In [45]:
import json
import math
import escher
import cobra
import cobrakbase
import modelseed_escher
from cobrakbase.core import KBaseFBAModel
from modelseed_escher.core import EscherMap

In [7]:
%run ../lib/kb_escher/kb_escher_utils.py

In [1]:
params = {
    'workspace_id': 37534, 
    'column_size': '4', 
    'report_height': '800', 
    'workspace_name': 'filipeliu:narrative_1580723870549', 
    'grid_maps': [
        {
            'object_ids': ['37534/91/1'], 
            'map_id': 'custom', 
            'user_map_id': '37534/38/1', 
            'model_id': '37534/90/3'
        }, 
        {
            'object_ids': ['37534/92/1'], 
            'map_id': 'FBA Checked TCA Cycle', 
            'user_map_id': '37534/88/1', 
            'model_id': '37534/89/1'
        }
    ]
}

In [35]:
token = "64XQ7SABQILQWSEW3CQKZXJA63DXZBGH"
api = cobrakbase.KBaseAPI(token, dev=True)

In [21]:
escher_seed = modelseed_escher.EscherManager(escher)

In [51]:
column_size = int(params['column_size'])
grid_cells = len(params['grid_maps'])
grid_size = (column_size, math.ceil(grid_cells / column_size))
grid_size

(4, 1)

In [48]:
em_list = []
for grid_cell in params['grid_maps']:
    em = build_grid_cell(grid_cell, escher_seed, api)
    em_list.append(em)

In [47]:
def build_grid_cell(grid_cell, escher_seed, api):
    em = None
    if not grid_cell['map_id'] == 'custom':
        em = escher_seed.get_map('ModelSEED', 'ModelSEED', grid_cell['map_id'])
    else:
        ref = api.get_object_info_from_ref(grid_cell['user_map_id'])
        map_data = api.get_object(ref.id, ref.workspace_id)
        em = modelseed_escher.core.EscherMap([map_data['metadata'], map_data['layout']])

    cmp_id = 'c0' #should come from parameters
    ref = api.get_object_info_from_ref(grid_cell['model_id'])
    fbamodel = KBaseFBAModel(api.get_object(ref.id, ref.workspace_id))
    alias = None
    if alias == None:
        alias = ref.id
    em = adapt_map_to_model(em, cmp_id, alias, fbamodel)
    return em

In [52]:
grid = modelseed_escher.EscherGrid()
grid_map = grid.build(em_list, grid_size)

In [ ]:
model_id = grid_block_data['model_id']

In [34]:
grid_cell['model_id']

'37534/89/1'

In [43]:
map_data.keys()

dict_keys(['layout', 'metadata'])

'/Users/fliu/Library/Caches/escher/1-0-0/5'